## **Advances in Data Mining**

Stephan van der Putten | (s1528459) | stvdputtenjur@gmail.com  
Theo Baart | s2370328 | s2370328@student.leidenuniv.nl

### **Assignment 1**
This assignment is concered with implementing formulas and models capable of predicting movie ratings for a set of users. Additionally, the accuracy of the various models are checked. 

Note all implementations are based on the assignment guidelines and helper files given as well as the documentation of the used functions.

#### **Model Approach**
This specific notebook handles the implementation of a Matrix factorization approach to the prediction problem.

In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt